In [2]:
from gensim.models import KeyedVectors
import json
with open('user_lookup.json', 'r') as f:
    user_lookup = json.load(f)

model = KeyedVectors.load_word2vec_format('GGvec_model_v1.bin')

In [3]:
import pandas as pd

In [4]:
positive_list = ['City:Lisbon']
selections_dict = {}
selections_dict['positive'] = positive_list

def get_recommendations(selections_dict, model, user_lookup, display_n=25):
    results = []
    for node_id, value in model.most_similar_cosmul(**selections_dict, topn=1000000):
        node_parts = node_id.split(':')
        if len(node_parts) != 2:
            node_type = node_parts[0]
            node_key = ' '.join(node_parts[1:])
        else:
            node_type, node_key = node_parts

        if node_type == 'Account':
            try:
                results.append((node_type, user_lookup[node_key]['Name'], value))
            except:
                print(node_key)
        elif node_key == 'True':
            pass
        else:
            results.append((node_type, node_id, value))
            
    df = pd.DataFrame(results, columns=['EntityType', 'Entity', 'Score'])
    return df.sort_values(by='Score',ascending=False)

In [5]:
res = get_recommendations(selections_dict, model, user_lookup, display_n=25)
res

,EntityType,Entity,Score
0,Account,carolicarvalho,0.795864
1,Account,MiguelOliveira88,0.794162
2,Account,∂iσgσ rєis,0.770878
3,Account,Margarida Louza,0.765715
4,Account,Rodrigo Polinicola,0.759210
...,...,...,...
24517,Brand,Brand:Moroccanoil,0.438353
24518,Account,stella andronikou,0.438159
24519,Account,Pablo Pecino,0.437473
24520,Country,Country:Spain,0.420344


In [7]:
import pprint
import pandas as pd
import json
import openai

pp = pprint.PrettyPrinter(indent=4)

token=''
organization = ''

openai.organization = organization
openai.api_key = token
engine_list = openai.Engine.list() # calling the engines available from the openai api 



In [9]:
content= '''
Here is the function which does cosine similarity. Now i am giving you instructions to get list of positive and negative entities in a natural form. your job is to give me to entities from the promt which has positive and negative entities in two separate list.

def get_recommendations(positive,negative, model, user_lookup, display_n=25):
    results = []
    for node_id, value in model.most_similar_cosmul(positive=list_of_positive,negative=list_of_negative, topn=100000):

        node_parts = node_id.split(':')
        if len(node_parts) != 2:
            node_type = node_parts[0]
            node_key = ' '.join(node_parts[1:])
        else:
            node_type, node_key = node_parts

        if node_type == 'Account':
            try:
                results.append((node_type, user_lookup[node_key]['Name'], value))
            except:
                print(node_key)
        elif node_key == 'True':
            pass
        else:
            results.append((node_type, node_id, value))

    df = pd.DataFrame(results, columns=['EntityType', 'Entity', 'Score'])

    results_tables = []
    for node_type, group_df in df.groupby('EntityType'):
        results_tables.extend([
            html.H4(node_type),
            dbc.Table.from_dataframe(group_df[:display_n], striped=True, bordered=True, hover=False)
        ])
        
    return html.Div(results_tables)
    

make sure to respond in following format only:

positive = ['something1','something2']
negative = ['something3','something4']

'''

def ask_llm_to_kg(prompt="I want to see similar user who are interested in coffee , lives in portugal and dont like beer"):
    MODEL = "gpt-4"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt},
        ],
        temperature=0.3

    )
    openai_resp = pp.pprint(response["choices"][0]["message"]['content'])
    response = response["choices"][0]["message"]['content'].split('\n')

    positive = eval(response[0].split('=')[1].strip())
    negative = eval(response[1].split('=')[1].strip())

    selections_dict={}
    selections_dict['positive'] = find_entity_invocab(positive)
    selections_dict['negative'] = find_entity_invocab(negative)
    print(selections_dict)
    res = get_recommendations(selections_dict, model, user_lookup, display_n=25)
    return res

In [12]:
# response = response["choices"][0]["message"]['content'].split('\n')
# positive = eval(response[0].split('=')[1].strip())
# negative = eval(response[1].split('=')[1].strip())


In [11]:
import re
vocab = list(model.vocab.keys())
# Define a regular expression to match entities


def find_entity_invocab(list_of_entities):
    entity_regex = re.compile(r"([\w%&'.()-`’]+):([\w%'-]+)")
    # Iterate over the terms and match against the entities in the vocab
    matched_entities = []
    for term in list_of_entities:
        # If the term does not match an entity, search for it in the vocab
        for entity in vocab:
            # if entity_regex.match(entity):
            #     entity_type, entity_num = entity_regex.match(entity).groups()
            #     if term.lower() in entity_num.lower():
            #         matched_entities.append(entity)
            if term.lower() in entity.split(':')[-1].lower():
                matched_entities.append(entity)

    # Print the matched entities
    print("Matched entities:", matched_entities)
    return matched_entities

In [14]:
# selections_dict={}
# selections_dict['positive'] = find_entity_invocab(positive)
# selections_dict['negative'] = find_entity_invocab(negative)
# selections_dict

In [15]:
# res = get_recommendations(selections_dict, model, user_lookup, display_n=25)
# res

In [16]:

def ask_llm_to_kg(prompt="I want to see similar user who are interested in coffee , lives in portugal and dont like beer"):
    MODEL = "gpt-4"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content":  prompt},
        ],
        temperature=0.3

    )
    openai_resp = pp.pprint(response["choices"][0]["message"]['content'])
    response = response["choices"][0]["message"]['content'].split('\n')

    positive = eval(response[0].split('=')[1].strip())
    negative = eval(response[1].split('=')[1].strip())

    positive_words =[]
    for phrase in positive:
        positive_words.extend(phrase.split())
    positive = list(set(positive_words))

    negative_words =[]
    for phrase in negative:
        negative_words.extend(phrase.split())
    negative = list(set(negative_words))


    selections_dict={}
    selections_dict['positive'] = find_entity_invocab(positive)
    selections_dict['negative'] = find_entity_invocab(negative)
    print(selections_dict)
    res = get_recommendations(selections_dict, model, user_lookup, display_n=25)
    return res

In [18]:
prompt = 'Show me who likes luxary products.'
ask_llm_to_kg(prompt)

"positive = ['luxury products', 'likes']\nnegative = []"
Matched entities: ['Interest:Luxury_Goods']
Matched entities: []
{'positive': ['Interest:Luxury_Goods'], 'negative': []}


,EntityType,Entity,Score
0,Brand,Brand:Gucci,0.660705
1,Brand,Brand:Christian_Dior,0.629836
2,Account,Kevin Pietersen 🦏,0.617124
3,Account,O B E S E S,0.613656
4,Account,Maorluz,0.612996
...,...,...,...
24517,Audience_Brand_Blogilates,Audience_Brand_Blogilates:Low,0.440246
24518,Audience_Brand_Trussardi,Audience_Brand_Trussardi:Low,0.436615
24519,Account,Mohamed Lyousfi 12⚽️,0.435882
24520,Audience_Brand_Dymatize,Audience_Brand_Dymatize:Low,0.430987


In [24]:
prompt = 'Show me who likes luxary products based out of Lisbon'
ask_llm_to_kg(prompt)

"positive = ['luxury products', 'Lisbon']\nnegative = []"
Matched entities: ['City:Lisbon', 'Interest:Luxury_Goods']
Matched entities: []
{'positive': ['City:Lisbon', 'Interest:Luxury_Goods'], 'negative': []}


,EntityType,Entity,Score
0,Account,Nuno Santos,0.388655
1,Account,MiguelOliveira88,0.387465
2,Account,Gonçalo 🥵,0.385126
3,Account,Laura Mourinho,0.385126
4,Account,carolicarvalho,0.380974
...,...,...,...
24516,Account,rene_ssaince,0.204266
24517,Account,bia_kellman,0.203589
24518,Account,Michelle Neukirchen,0.203523
24519,Audience_Brand_Campari,Audience_Brand_Campari:High,0.202570


In [17]:
prompt = 'Show me who are into drinking coffee and like active lifestyle, dont like to drink beer or smoking'
ask_llm_to_kg(prompt)

("positive = ['drinking coffee', 'active lifestyle']\n"
 "negative = ['drinking beer', 'smoking']")
Matched entities: ['Brand:Coffeegram', 'Brand:Costa_Coffee', 'Interest:Coffee,_Tea_&_Beverages', 'Interest:Healthy_Lifestyle', 'Interest:Lifestyle', 'Interest:Activewear']
Matched entities: ['Interest:Beer,_Wine_&_Spirits', 'Interest:Tobacco_&_Smoking']
{'positive': ['Brand:Coffeegram', 'Brand:Costa_Coffee', 'Interest:Coffee,_Tea_&_Beverages', 'Interest:Healthy_Lifestyle', 'Interest:Lifestyle', 'Interest:Activewear'], 'negative': ['Interest:Beer,_Wine_&_Spirits', 'Interest:Tobacco_&_Smoking']}


,EntityType,Entity,Score
0,Account,Travel&Food 🌍 Ebru,0.196123
1,Account,DsyMkn🐝,0.179381
2,Account,Nothing Beats Love,0.179016
3,Account,Karina 🇱🇹,0.178200
4,Account,Hotel Granduca,0.177151
...,...,...,...
24510,Account,zack rosebrugh,0.032733
24511,Account,𝕷𝖔𝖑𝖆 𝕾𝖆́𝖓𝖈𝖍𝖊𝖟,0.032587
24512,Account,Celtian,0.032271
24513,Account,jip,0.031082
